### Data Preparation

Clustering, Dimensionality Reduction, then Both, with clustering evaluation


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA, FastICA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import learning_curve
from sklearn.impute import SimpleImputer
import joblib
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Load the metadata
metadata = pd.read_csv("/kaggle/input/isic-2024-challenge/train-metadata.csv", low_memory=False)

# Use only 5% of the data for now
metadata_sampled = metadata.sample(frac=0.05, random_state=42)

# Preprocess the metadata
def preprocess_metadata(metadata, is_train=True):
    # Fill missing values
    metadata = metadata.fillna(metadata.median(numeric_only=True))
    metadata = metadata.fillna('Unknown')

    # Columns specific to training data
    train_only_cols = ['target', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5']
    drop_cols = ['isic_id', 'patient_id']

    # Drop columns specific to training data
    if is_train:
        drop_cols.extend(train_only_cols)
    
    metadata = metadata.drop(columns=[col for col in drop_cols if col in metadata.columns])

    # Define categorical and numerical columns
    categorical_cols = metadata.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = metadata.select_dtypes(include=['number']).columns.tolist()

    # One-hot encode categorical variables and scale numerical variables
    preprocessor = ColumnTransformer(
        transformers=[
            ('num_imputer', SimpleImputer(strategy='mean'), numerical_cols),
            ('num_scaler', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_cols)])

    if is_train:
        return preprocessor.fit(metadata)
    else:
        return preprocessor.transform(metadata)

# Preprocess the metadata
preprocessor = preprocess_metadata(metadata_sampled, is_train=True)
metadata_processed = preprocessor.transform(metadata_sampled)
target = metadata_sampled['target']

# Save the preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')

# Dimensionality reduction and clustering functions
def apply_dimensionality_reduction(X, method, n_components=2):
    if method == 'PCA':
        dr = PCA(n_components=n_components)
    elif method == 'ICA':
        dr = FastICA(n_components=n_components)
    elif method == 'RP':
        dr = GaussianRandomProjection(n_components=n_components)
    elif method == 't-SNE':
        perplexity = min(30, X.shape[0] - 1)
        dr = TSNE(n_components=n_components, perplexity=perplexity)
    else:
        raise ValueError(f"Unknown method: {method}")
    X_reduced = dr.fit_transform(X)
    return X_reduced

def apply_clustering(X, method, n_clusters=2):
    if method == 'KMeans':
        model = KMeans(n_clusters=n_clusters, random_state=42)
    elif method == 'GMM':
        model = GaussianMixture(n_components=n_clusters, random_state=42)
    else:
        raise ValueError(f"Unknown clustering method: {method}")
    clusters = model.fit_predict(X)
    return clusters

# Plotting functions
def plot_visualization_grid(ax, method, X, y=None):
    X_reduced = apply_dimensionality_reduction(X, method)
    if y is not None:
        ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, cmap='viridis', edgecolor='k')
    else:
        ax.scatter(X_reduced[:, 0], X_reduced[:, 1], cmap='viridis', edgecolor='k')
    ax.set_title(f"{method} Visualization")

def plot_clustering_grid(ax, method, X):
    clusters = apply_clustering(X, method)
    silhouette_avg = silhouette_score(X, clusters)
    ax.scatter(X[:, 0], X[:, 1], c=clusters, cmap='viridis', edgecolor='k')
    ax.set_title(f"{method} Clustering (Silhouette: {silhouette_avg:.2f})")

def plot_dim_red_then_clustering(ax, dim_red_method, cluster_method, X):
    X_reduced = apply_dimensionality_reduction(X, dim_red_method)
    clusters = apply_clustering(X_reduced, cluster_method)
    silhouette_avg = silhouette_score(X_reduced, clusters)
    ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=clusters, cmap='viridis', edgecolor='k')
    ax.set_title(f"{dim_red_method} + {cluster_method} (Silhouette: {silhouette_avg:.2f})")

# Plot in grid
def plot_in_grid(methods, plot_function, rows, cols, title_prefix, X, y=None, is_clustering=False):
    fig, axs = plt.subplots(rows, cols, figsize=(20, 10))
    axs = axs.flatten()
    for i, method in enumerate(methods):
        if is_clustering:
            plot_function(axs[i], method, X)
        else:
            plot_function(axs[i], method, X, y)
    for ax in axs[len(methods):]:
        fig.delaxes(ax)
    plt.tight_layout()
    plt.suptitle(f"{title_prefix}", y=1.02, fontsize=16)
    plt.show()

# Dimensionality reduction then clustering
def plot_dim_red_then_clustering_grid(dim_red_methods, cluster_methods, X):
    fig, axs = plt.subplots(len(dim_red_methods), len(cluster_methods), figsize=(20, 5 * len(dim_red_methods)))
    for i, dim_red_method in enumerate(dim_red_methods):
        for j, cluster_method in enumerate(cluster_methods):
            plot_dim_red_then_clustering(axs[i, j], dim_red_method, cluster_method, X)
    plt.tight_layout()
    plt.suptitle("Dim. Reduction + Clustering", y=1.02, fontsize=16)
    plt.show()

# Apply and plot
clustering_methods = ['KMeans', 'GMM']
dim_red_methods = ['PCA', 'ICA', 'RP', 't-SNE']

# Step 1: Run clustering, then visualize data
plot_in_grid(clustering_methods, plot_clustering_grid, 1, 2, "Clustering Methods", metadata_processed, y=None, is_clustering=True)

# Step 2: Run dim. red. then visualize data
plot_in_grid(dim_red_methods, plot_visualization_grid, 2, 2, "Dim. Red. Methods", metadata_processed, y=None, is_clustering=False)

# Step 3: Run dim. red., then clustering, then visualize data
plot_dim_red_then_clustering_grid(dim_red_methods, clustering_methods, metadata_processed)

# Metrics evaluation functions
def evaluate_pca(X):
    pca = PCA()
    pca.fit(X)
    explained_variance = pca.explained_variance_ratio_
    return explained_variance

def evaluate_ica(X):
    ica = FastICA()
    ica.fit(X)
    kurtosis = pd.DataFrame(ica.components_).kurt(axis=1).abs().values
    return kurtosis

def evaluate_rp(X):
    rp = GaussianRandomProjection(n_components=50)
    X_projected = rp.fit_transform(X)
    X_reconstructed = np.dot(X_projected, np.linalg.pinv(rp.components_.T))
    reconstruction_error = np.mean((X - X_reconstructed) ** 2)
    return reconstruction_error

def tabulate_metrics(X):
    pca_var = evaluate_pca(X)
    ica_kurt = evaluate_ica(X)
    rp_err = evaluate_rp(X)
    
    print("PCA Explained Variance Ratio:", pca_var)
    print("ICA Mean Kurtosis:", ica_kurt)
    print("RP Reconstruction Error:", rp_err)

# Tabulate metrics
tabulate_metrics(metadata_processed)

# Prepare data for the model
X_train, X_val, y_train, y_val = train_test_split(metadata_processed, target, test_size=0.2, random_state=42)

# Define and train the MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Save the trained model and the dimensionality reduction method used
joblib.dump(mlp, 'mlp_model.pkl')
joblib.dump('PCA', 'dim_red_method.pkl')  # Example, you should save the actual method you used

# Predict and evaluate
y_pred = mlp.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred)

print(f"ROC AUC Score: {roc_auc}")

# Plot ROC Curve
fpr, tpr, _ = roc_curve(y_val, y_pred)
plt.figure()
plt.plot(fpr, tpr, label=f'MLP (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot Learning Curves
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 5), scoring='roc_auc'):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score" if scoring == 'roc_auc' else "Error rate")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring
    )
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()
    
    if scoring == 'roc_auc':
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    else:
        train_error = 1 - train_scores_mean
        test_error = 1 - test_scores_mean
        plt.fill_between(train_sizes, train_error - train_scores_std,
                         train_error + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_error - test_scores_std,
                         test_error + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_error, 'o-', color="r", label="Training error")
        plt.plot(train_sizes, test_error, 'o-', color="g", label="Cross-validation error")

    plt.legend(loc="best")
    return plt

# Plot learning curve for ROC AUC
plot_learning_curve(mlp, "Learning Curve (ROC AUC) - MLP", X_train, y_train, cv=5, scoring='roc_auc')
plt.show()

# Plot learning curve for error (1 - accuracy)
plot_learning_curve(mlp, "Learning Curve (Error) - MLP", X_train, y_train, cv=5, scoring='accuracy')
plt.show()


### Model Training and Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import learning_curve

# Define and train the MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Predict and evaluate
y_pred = mlp.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred)

print(f"ROC AUC Score: {roc_auc}")

# Plot ROC Curve
fpr, tpr, _ = roc_curve(y_val, y_pred)
plt.figure()
plt.plot(fpr, tpr, label=f'MLP (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot Learning Curves
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 5), scoring='roc_auc'):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score" if scoring == 'roc_auc' else "Error rate")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring
    )
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()
    
    if scoring == 'roc_auc':
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    else:
        train_error = 1 - train_scores_mean
        test_error = 1 - test_scores_mean
        plt.fill_between(train_sizes, train_error - train_scores_std,
                         train_error + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_error - test_scores_std,
                         test_error + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_error, 'o-', color="r", label="Training error")
        plt.plot(train_sizes, test_error, 'o-', color="g", label="Cross-validation error")

    plt.legend(loc="best")
    return plt

# Plot learning curve for ROC AUC
plot_learning_curve(mlp, "Learning Curve (ROC AUC) - MLP", X_train, y_train, cv=5, scoring='roc_auc')
plt.show()

# Plot learning curve for error (1 - accuracy)
plot_learning_curve(mlp, "Learning Curve (Error) - MLP", X_train, y_train, cv=5, scoring='accuracy')
plt.show()


### Inference Test